# Laboratorio 4
## Universidad del Valle de Guatemala <br> Facultad de Ingeniería
#### Departamento de Ciencias de la Computación <br> Data Science - Sección 10

In [77]:
# Importar librerías
import pandas as pd
import nltk
import re

In [78]:
from nltk import bigrams
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [79]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/crislay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/crislay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [81]:
stop_words = set(stopwords.words('english'))

#### 2. Cargar el archivo de datos.

In [82]:
# 1. Ya se descargo el archivo train.csv
data_frame = pd.read_csv('./data/train.csv')
data_frame

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Se puede apreciar que se tienen las siguientes variables junto con su descripción y estado:

- id: Este sería el identificador del texto
- keyword: Palabra clave de como "catagolaron" el tweet
- location: El lugar en donde se publico el tewt
- text: El texto del mismo tweet publicado
- target: Clasificacion del tweet si es un desastre real o no

Se puede apreciar que se pueden encontrar datos vacios en keyword y en location.

#### 3. Limpie y preprocese los datos (Describir de forma detallada las actividades de preprocesamiento que se llevó a cabo)

In [83]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

In [84]:
stop_words = set(['the', 'a', 'an', 'in', 'at', 'is', 'it', 'this', 'that', 'on', 'was', 'by', 'for', 'with', 'as', 'and', 'or', 'to', 'from', 'of'])

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [85]:
# Convertir el texto en minúsculas
data_frame['text'] = data_frame['text'].str.lower()

# Quitar caracteres especiales “#”,”@” o los apóstrofes
data_frame['text'] = data_frame['text'].str.replace(r'[#@\'"]', '', regex=True)

# Quitar urls
data_frame['text'] = data_frame['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', regex=True)

# Eliminar signos de puntuacion
data_frame['text'] = data_frame['text'].str.replace(r'[^\w\s]', '', regex=True)

In [86]:
# Revisar si hay emojis en el texto y luego quitarlos
data_frame['text'] = data_frame['text'].apply(remove_emojis)

In [87]:
# Quitar los artículos, preposiciones y conjunciones (stopwords)
data_frame['text'] = data_frame['text'].apply(remove_stopwords)

In [88]:
data_frame

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are reason earthquake may allah forg...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked shelter place are being no...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,just got sent photo ruby alaska smoke wildfire...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding bridge collapse into ...,1
7609,10870,NaN,NaN,aria_ahrary thetawniest out control wild fires...,1
7610,10871,NaN,NaN,m194 0104 utc5km s volcano hawaii,1
7611,10872,NaN,NaN,police investigating after ebike collided car ...,1


Como se podrá observar se opto por quitar convertir todo el te xto en minusculas, en quitar caracteres especiales, removemoer urls, eliminar signos de puntuacion extraños, luego quitar todos los emojis y luego remover los "stopwords" el cual en consecuencia nos permite tener data mejor procesada. Se tomara en cuetna el tipo de keywrod y location que tiene para apoyarnos en detectar las palabras usadas con más frecuencia y aparte detectar si en efecto se hablan de desastres reales o no.

#### 4. Obtener frecuencia de las palabras tanto de los tweets de desastres como de los que no.

In [89]:
# Separar los tweets de desastres y los que no lo son
disasters_tweets = data_frame[data_frame['target'] == 1]['text']
no_disasters_tweets = data_frame[data_frame['target'] == 0]['text']

# Tokenizar los tweets
tokens_disasters = word_tokenize(' '.join(disasters_tweets))
tokens_no_disasters = word_tokenize(' '.join(no_disasters_tweets))

# Calcular la frecuencia de las palabras
freq_disasters = FreqDist(tokens_disasters)
freq_no_disasters = FreqDist(tokens_no_disasters)

In [90]:
# Imprimir las palabras más comunes
print('Palabras más comunes en tweets de desastres:')
for word, frequency in freq_disasters.most_common(20):
    print(f'{word}: {frequency}')

Palabras más comunes en tweets de desastres:
i: 294
after: 181
fire: 178
are: 166
have: 142
news: 136
my: 131
you: 129
over: 127
up: 126
more: 122
via: 121
were: 119
disaster: 117
be: 114
california: 111
suicide: 110
no: 107
police: 107
amp: 106


In [91]:
print('\nPalabras más comunes en tweets que no son de desastres:')
for word, frequency in freq_no_disasters.most_common(20):
    print(f'{word}: {frequency}')


Palabras más comunes en tweets que no son de desastres:
i: 1076
you: 664
my: 544
be: 287
me: 260
like: 253
im: 243
have: 242
so: 240
are: 235
your: 231
just: 231
but: 220
not: 209
up: 195
out: 195
amp: 192
all: 185
if: 183
will: 179


##### ¿Qué palabras cree que le servirán para hacer un mejor modelo de clasificación?

Caso de tweets de desastres:
- after
- fire
- news
- over
- more
- suicide
- police
- amp

En lo personal creo que estas serian las mejores palabras para clasificar un desastre real ya que estos tocan temas serios.

Caso de tweets de no desastres:
- my
- be
- me
- like
- your
- just
- if
- will

Para el caso de los no desastres considero que estas palabras serían mejores para clasificar los temas de ironia y comedia, ya que como se puede apreciar son palabras haciendo referencia a una persona, a algunos temas agradables e inclusive hablando sobre "que pasaría".

##### ¿Vale la pena explorar bigramas o trigramas para analizar contexto?

##### Vamos a obtener y crear los bigramas

In [92]:
# Crear bigramas
bigrams_disasters = list(bigrams(tokens_disasters))
bigrams_no_disasters = list(bigrams(tokens_no_disasters))

# Calcular la frecuencia de los bigramas
freq_bigrams_disasters = FreqDist(bigrams_disasters)
freq_bigrams_no_disasters = FreqDist(bigrams_no_disasters)

In [93]:
# Imprimir los bigramas más comunes
print('Bigramas más comunes en tweets de desastres:')
for word, frequency in freq_bigrams_disasters.most_common(15):
    print(f'{word}: {frequency}')

Bigramas más comunes en tweets de desastres:
('suicide', 'bomber'): 59
('more', 'than'): 43
('have', 'been'): 42
('northern', 'california'): 41
('oil', 'spill'): 38
('suicide', 'bombing'): 34
('california', 'wildfire'): 34
('more', 'homes'): 32
('may', 'have'): 31
('70', 'years'): 30
('homes', 'razed'): 29
('confirmed', 'mh370'): 28
('latest', 'more'): 28
('razed', 'northern'): 28
('16yr', 'old'): 28


In [94]:
print('\nBigramas más comunes en tweets que no son de desastres:')
for word, frequency in freq_bigrams_no_disasters.most_common(15):
    print(f'{word}: {frequency}')


Bigramas más comunes en tweets que no son de desastres:
('i', 'just'): 55
('if', 'you'): 48
('i', 'have'): 44
('i', 'dont'): 43
('you', 'are'): 38
('will', 'be'): 38
('cross', 'body'): 38
('youtube', 'video'): 36
('i', 'liked'): 35
('liked', 'youtube'): 35
('do', 'you'): 34
('i', 'cant'): 32
('gon', 'na'): 32
('wan', 'na'): 30
('i', 'will'): 29


##### Ahora vamos a crear trigramas para analizar los diferentes contextos, se utilizaran 3 n-gramas para calcular los trigramas.

In [95]:
# Crear trigramas
trigrams_disasters = list(ngrams(tokens_disasters, 3))
trigrams_no_disasters = list(ngrams(tokens_no_disasters, 3))

# Calcular la frecuencia de los trigramas
freq_trigrams_disasters = FreqDist(trigrams_disasters)
freq_trigrams_no_disasters = FreqDist(trigrams_no_disasters)

In [103]:
# Imprimir los trigramas más comunes
print('Trigramas más comunes en tweets de desastres:')
for word, frequency in freq_trigrams_disasters.most_common(15):
    print(f'{word}: {frequency}')

Trigramas más comunes en tweets de desastres:
('northern', 'california', 'wildfire'): 29
('more', 'homes', 'razed'): 29
('latest', 'more', 'homes'): 28
('homes', 'razed', 'northern'): 28
('pkk', 'suicide', 'bomber'): 28
('suicide', 'bomber', 'who'): 28
('bomber', 'who', 'detonated'): 28
('who', 'detonated', 'bomb'): 28
('razed', 'northern', 'california'): 27
('16yr', 'old', 'pkk'): 27
('old', 'pkk', 'suicide'): 27
('families', 'sue', 'over'): 26
('sue', 'over', 'legionnaires'): 26
('more', 'than', '40'): 26
('than', '40', 'families'): 26


In [104]:
print('\nTrigramas más comunes en tweets que no son de desastres:')
for word, frequency in freq_trigrams_no_disasters.most_common(15):
    print(f'{word}: {frequency}')


Trigramas más comunes en tweets que no son de desastres:
('i', 'liked', 'youtube'): 35
('liked', 'youtube', 'video'): 35
('reddit', 'will', 'now'): 21
('will', 'now', 'quarantine'): 21
('cross', 'body', 'bag'): 18
('now', 'quarantine', 'offensive'): 18
('quarantine', 'offensive', 'content'): 18
('my', 'pick', 'fan'): 17
('pick', 'fan', 'army'): 17
('reddits', 'new', 'content'): 16
('new', 'content', 'policy'): 16
('stock', 'market', 'crash'): 16
('full', 'read', 'ebay'): 15
('fall', 'off', 'cliff'): 15
('ignition', 'knock', 'detonation'): 15


En base a los resultados obtenidos se puede apreciar que vale más la pena utilizar los trigramas ya que estos suelen dar mucho más contexto que los bigramas, los bigramas algunos de sus resultados pueden dejar muy ambiguos el contexto y se puede interpretar de formas muy diferentes. Sin emmbargo con los trigramas se combinan palabras más serias y dan más contexto ya sea un desastre real o no.

In [110]:
# Modelo de predicción y definir el algoritmo a utilizar

#### 5. Realizar un análisis exploratorio de los datos para entrenderlos mejor, documente todos los análisis.

In [105]:
# Investigar qué palabra se repite más en cada una de las categorías

In [106]:
# Hacer una nube de palabras para visualizar las que aparecen con más frecuencia

In [107]:
# Hacer un histograma con las palabras que más se repiten

In [108]:
# Discutir sobre las palabras que tienen presencia en todas las categorías.

In [ ]:
# ...
# ! explicar los cruces de variables, hay gráficos
# ! explicativos y análisis que permiten comprender el conjunto de datos

#### 6. Elabore una función en el que el usuario ingrese un tweet y el sistema lo clasifica en desastre o no.

In [109]:
# Funcion para clasificar los nuevos tweets a ingresar